In [1]:
import os
import sys
import json
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

# Setup stuff.  The cell just supports the workbook, you can ignore it
EXAMPLE_DO_FOLDER = os.path.join(os.getcwd(), "example_do_folder")
INSERTION_FOLDER = os.path.join(EXAMPLE_DO_FOLDER, "hello/do_examples")

def show(base):
    with open(do.get_base(base).__file__) as file:
        contents = file.read()
    with_bar = "\n  | ".join(contents.split("\n"))
    print(F"\n### SHOWING MODULE {'.../'+base+'.py'!r}\n  | {with_bar}\n\n")

def run_do(*args, **kwargs):
    parts = [repr(x) for x in args] + [F"{k}={v!r}" for k, v in kwargs.items()]
    print(F"do({', '.join(parts)})")
    result = do(*args, **kwargs)
    print(F"--> {result!r}")
    print()


print(F"### INSERTION FOLDER        = {INSERTION_FOLDER!r}")
print(F"### DO FOLDER (in Jupyter)  = {EXAMPLE_DO_FOLDER}")
sys.path.append(os.path.dirname(os.path.dirname(EXAMPLE_DO_FOLDER)))
from dvc_dat import do   # Add all loadables BEFORE loading this module
print(F"## DO FOLDER (in do module) = {do.do_folder!r}")
if EXAMPLE_DO_FOLDER != do.do_folder:
    print(f"WARNING: EXAMPLE_DO_FOLDER used by jupyter = {EXAMPLE_DO_FOLDER} does not match {do.do_folder}")
if not os.path.exists(INSERTION_FOLDER):
    input(f"WARNING: INSERTION_FOLDER {INSERTION_FOLDER!r} not found.")
    os.makedirs(INSERTION_FOLDER)
print("\n\n\n\n")


### INSERTION FOLDER        = '/Users/oblinger/ob/proj/dvc-dat/examples/example_do_folder/hello/do_examples'
### DO FOLDER (in Jupyter)  = /Users/oblinger/ob/proj/dvc-dat/examples/example_do_folder
## DO FOLDER (in do module) = '/Users/oblinger/ob/proj/dvc-dat/examples/example_do_folder'







# Example Data

### Imports

In [2]:
from dvc_dat import dat_tools as dt
from dvc_dat import Dat, do

### Some Example Dats
These Dat and point_fns are used to build and show DataFrames.


In [3]:
TMP_PATH1 = "/tmp/job_test1"
spec1 = {"dat": {"my_key1": "my_val1", "my_key2": "my_val2"}}
dat1 = Dat.create(spec=spec1, path=TMP_PATH1, overwrite=True)

TMP_PATH2 = "/tmp/job_test2"
spec2 = {"dat": {"my_key1": "my_val1", "my_key2": "my_val2"}, "other": "key_value"}
dat2 = Dat.create(spec=spec2, path=TMP_PATH2, overwrite=True)


.

.

## Creating DataFrames from Dats and point_fns

#### Zero point_fns applied to zero Dats yields an empty DataFrame

In [4]:
dt.from_dat([], [])  # Cross-product of an empty list of dats and empty list of do functions is an empty DataFrames

""


#### Two metrics applied to two dats yields a DataFrame w/ two rows & two cols

In [5]:
def always_17(_dat):
    return 17


def always_18(_dat):
    return 18

dt.from_dat([dat1, dat2], [always_17, always_18])

,always_17,always_18,list
0,17,18,job_test1
1,17,18,job_test2


#### Metric functions can also be expressed as string and loaded using 'do'

In [6]:
do.mount(at="registered_cube", module="example_do_folder.dat_tools_examples.cube_hello")
dt.from_dat([dat1], ["registered_cube.always_5"])

,always_5,list
0,5,job_test1


#### Multi-valued Metric Functions
Metric functions (also called point_fns) can return a dict of metric values instead of just returning a single value.

In [7]:
fns = [always_17, lambda dat: {"val1": 111, "val2": 2222}]
dt.from_dat([dat1, dat2], point_fns=fns)

,always_17,val1,val2,list
0,17,111,2222,job_test1
1,17,111,2222,job_test2


#### Multi-POINT Metric Functions
Finally, a point_fn can return a LIST of dicts, each dict represents a row in the DataFrame.  In all cases, all point_fns are applied to all dats, and the return values are appended to the DataFrame.  These different kinds of point functions can be combined, so scalar values and dict values are all merged into a single row for each dat, and lists of dicts are expanded into multiple rows.

In [8]:
point_fns = [always_17, always_18, lambda dat: [{"val1": 1, "val2": 2}, {"val1": 11}]]
dt.from_dat([dat1], point_fns)

,val1,val2,list,always_17,always_18
0,1.0,2.0,job_test1,NaN,NaN
1,11.0,NaN,job_test1,NaN,NaN
2,NaN,NaN,job_test1,17.0,18.0


Above, you can see that the results of the always_17 and always_18 functions were merged into a single row as before, and the lambda returned to more data points expressed as two additional rows in the DataFrame.

# OUTPUT TO EXCEL
The 'to_excel' function writes the DataFrame to an Excel file.
To test this, we use the points_fn below 'load_points_json' it reads a list of data points from each dat by looking in the dat folder for a file named 'points.json'.  This is used to construct a large DataFrame from three Dats in the "Retail Data" DatContainer.

In [9]:
def load_points_json(dat: Dat):
    with open(f"{dat.get_path()}/points.json") as f:
        return json.load(f)


In [10]:
df = dt.from_dat("Datasets/Retail Data", [load_points_json])
df

,Store,Month,Product,Metric,Value,list
0,Store A,January,Product 1,Units Sold,8,Berkeley
1,Store A,January,Product 1,Revenue,23,Berkeley
2,Store A,January,Product 2,Units Sold,80,Berkeley
3,Store A,January,Product 2,Revenue,88,Berkeley
4,Store A,February,Product 1,Units Sold,76,Berkeley
5,Store A,February,Product 1,Revenue,18,Berkeley
6,Store A,February,Product 2,Units Sold,54,Berkeley
7,Store A,February,Product 2,Revenue,98,Berkeley
8,Store B,January,Product 1,Units Sold,51,Berkeley
9,Store B,January,Product 1,Revenue,25,Berkeley


In [11]:
dt.to_excel(df, show=True)

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx


'/Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx'

#### The 'sheets' parameter
The 'sheets' to_excel parameter can be used to split a data frame into separate Excel sheets according to the value of the indicated column.  Here we split the DataFrame so each city is on its own sheet.

In [12]:
dt.to_excel(df, sheets=["list"], show=True)

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx


'/Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx'

Providing multiple columns in the 'sheets' parameter will create a sheets for the cross-product of column values as shown here:

In [13]:
dt.to_excel(df, sheets=["list", "Store"], show=True)

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx


'/Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx'

#### The 'docs' parameter
Another way to slice a DataFrame is into separate Excel documents.  Here we split the DataFrame by month:

In [14]:
dt.to_excel(df, docs=["Month"], show=True)

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/February.xlsx
# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/January.xlsx


'/Users/oblinger/ob/proj/dvc-dat/examples'

#### The 'add_formatted_column' parameter
Often a report can be tidied up by combining related columns into single semantically
unified ones.   The 'add_formatted_columns' defines a simple formatting syntax based
on Python's format fn. Here we combine the 'Metric' and 'Value' columns into the new
'Result' column.

In [15]:
dt.to_excel(df, sheets=["Store", "Month"], show=True,
            formatted_columns=["Result <== {0} {1} <==  Value, Metric"])

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx


'/Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx'

#### The 'columns' parameter
The 'columns' parameter also can be used to trim the Excel sheet(s) to only include the listed
columns.  This can also be used to "tidy up" the output.

In [16]:
dt.to_excel(df, sheets=["Store", "Month"], show=True,
            columns=["list", "Product", "Result"],
            formatted_columns=["Result <== {0} {1} <==  Value, Metric"])

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx


'/Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx'

#### The 'transform' parameter
The formatted_columns parameter was included as combining columns is a common
special case.  More generally, one can use the 'transform' parameter to perform
arbitrary transformation of the dataframe before writing it to Excel.  It accepts a
transform function that returns a new transformed DataFrame based on its input DF.

Here we have a transform fn that computes average revenue and average units sold then
conditionally adds respective percentage calculations after dollars or units sold as
appropriate.

In [17]:
def my_transform(df_):
    """Adds a 'Result' column to the DataFrame.
    
    Computes two sums, one for 'Revenue' and one for 'Units Sold' then 
    computes percentage relative to the average for each metric."""
    def my_result(row):
        if row['Metric'] == 'Revenue':
            percentage = (row['Value'] / average_revenue) * 100
            return f"${row['Value']} ({int(percentage)}%)"
        elif row['Metric'] == 'Units Sold':
            percentage = (row['Value'] / average_units) * 100
            return f"{row['Value']} Units ({int(percentage)}%)"
        else:
            return "???"
    average_revenue = df_[df_['Metric'] == 'Revenue']['Value'].mean()
    average_units = df_[df_['Metric'] == 'Units Sold']['Value'].mean()
    df_['Result'] = df_.apply(my_result, axis=1)
    return df_

dt.to_excel(df, sheets=["Store", "Month"], show=True,
            columns=["list", "Product", "Result"],
            transform=my_transform)

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx


'/Users/oblinger/ob/proj/dvc-dat/examples/output.xlsx'

#### Combining many parameters
Here we split into docs and sheets, applies transform, and selects specific columns

And we can combine these methods to slice the data simultaneously by document and sheet:

In [18]:
dt.to_excel(df, show=True,
            title="Tidy Retail Report",
            folder="/tmp",
            docs=["Month"],
            sheets=["list", "Store"],
            columns=["Product", "Result"],
            transform=my_transform)

# Dataframe written to /tmp/Tidy Retail Report February.xlsx
# Dataframe written to /tmp/Tidy Retail Report January.xlsx


'/tmp'

# METRICS_MATRIX
The 'dat_report' function is used to create a matrix of metrics from a list of
Dats and a list of metric functions.  The matrix is a 2D array where each row
represents a Dat, and each column represents a metric function.  The 'dat_report'
function is useful for creating a matrix of metrics for a set of Dats that can be used
for further analysis or visualization.

The 'dat_report' is designed to be used as a do-method.


In [19]:
show("mr_sprint23")


### SHOWING MODULE '.../mr_sprint23.py'
  | __main__ = {
  |     "dat": {"do": "dt.dat_report"},
  |     "dat_report": {
  |         "title": "S23",
  |         "source": ["runs/example/hello10", "runs/example/hello5"],
  |         "metrics": ["cube_hello.is_prime", "cube_hello.data", "cube_hello.color_p"],
  |         "show": True}
  | }
  | 
  | add_col = {
  |     "dat": {"base": "mr_sprint23"},
  |     "dat_report": {
  |         "title": "S23b", "show": False, "verbose": False,
  |         "formatted_columns":
  |             [" new_col <== {0}-{1} <== is_prime, color_p "]}
  | }
  | 




In [27]:
do("mr_sprint23")

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/S23.xlsx


,is_prime,data,color_p,list
0,False,6-11-4-7-5-8-2-11-3-2,0.4,0
1,False,6-8-2-3-3-3-3-5-3-2,0.4,1
2,True,0-1-2-1-2-3-0-2-3-3,0.4,2
3,True,8-7-9-11-7-8-8-9-10-7,0.4,3
4,False,11-11-11-11-11-11-10-10-10-11,0.4,4
5,True,9-8-11-14-13-8-7-8-13-6,0.4,5
6,False,3-4-4-6-3-6-2-8-6-8,0.4,6
7,True,10-10-10-10-11-10-11-10-10-10,0.4,7
8,False,9-16-9-6-7-11-11-11-16-11,0.4,8
9,False,7-10-4-8-8-5-5-8-4-9,0.4,9


#### Do commands all exported to command line
All do commands can also be run from the command line.  The 'do' launcher script:

In [28]:
!./do mr_sprint23

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/S23.xlsx
    is_prime                           data  color_p list
0      False          6-11-4-7-5-8-2-11-3-2      0.4    0
1      False            6-8-2-3-3-3-3-5-3-2      0.4    1
2       True            0-1-2-1-2-3-0-2-3-3      0.4    2
3       True          8-7-9-11-7-8-8-9-10-7      0.4    3
4      False  11-11-11-11-11-11-10-10-10-11      0.4    4
5       True        9-8-11-14-13-8-7-8-13-6      0.4    5
6      False            3-4-4-6-3-6-2-8-6-8      0.4    6
7       True  10-10-10-10-11-10-11-10-10-10      0.4    7
8      False      9-16-9-6-7-11-11-11-16-11      0.4    8
9      False           7-10-4-8-8-5-5-8-4-9      0.4    9
10     False   12-10-12-11-13-14-14-10-14-9      0.4    0
11     False            1-4-6-4-4-2-6-6-4-8      0.4    1
12      True  10-10-10-10-10-10-10-10-10-10      0.4    2
13      True      11-11-13-12-10-9-9-9-13-5      0.4    3
14     False            5-5-5-5-5-5-4-5-5-5      0.4    4

#### Configuration inheritance
Configuration shadowing inheritance eases testing of special cases.  Here we add a column to the 'mr_sprint23' configuration.

In [29]:
!./do mr_sprint23.add_col

# Dataframe written to /Users/oblinger/ob/proj/dvc-dat/examples/S23b.xlsx
    is_prime                           data  color_p list    new_col
0      False          6-11-4-7-5-8-2-11-3-2      0.4    0  False-0.4
1      False            6-8-2-3-3-3-3-5-3-2      0.4    1  False-0.4
2       True            0-1-2-1-2-3-0-2-3-3      0.4    2   True-0.4
3       True          8-7-9-11-7-8-8-9-10-7      0.4    3   True-0.4
4      False  11-11-11-11-11-11-10-10-10-11      0.4    4  False-0.4
5       True        9-8-11-14-13-8-7-8-13-6      0.4    5   True-0.4
6      False            3-4-4-6-3-6-2-8-6-8      0.4    6  False-0.4
7       True  10-10-10-10-11-10-11-10-10-10      0.4    7   True-0.4
8      False      9-16-9-6-7-11-11-11-16-11      0.4    8  False-0.4
9      False           7-10-4-8-8-5-5-8-4-9      0.4    9  False-0.4
10     False   12-10-12-11-13-14-14-10-14-9      0.4    0  False-0.4
11     False            1-4-6-4-4-2-6-6-4-8      0.4    1  False-0.4
12      True  10-10-10-10-10-

#### Combining configuration and code
Allowing configuration structures to be defined directly in python allows configuration
to be combined with code in the same file.  Combining code and configuration in this
way is powerful, natural, and simple.

In [31]:
#### Cleanup
!echo CWD = $PWD
!ls *.xlsx
!rm *.xlsx
!rm -r example_dat_data_folder/runs/doubler example_dat_data_folder/runs/mspipe

CWD = /Users/oblinger/ob/proj/dvc-dat/examples
zsh:1: no matches found: *.xlsx
zsh:1: no matches found: *.xlsx
